# Demo of XAI for sentiment analysis

This notebook consists of calculation of attribution using Integrated Gradients method for siebert model. It was inspired by and based on the official tutorial: [https://captum.ai/tutorials/Bert_SQUAD_Interpret](https://captum.ai/tutorials/Bert_SQUAD_Interpret)

In [1]:
from transformers import pipeline
from captum.attr import LayerIntegratedGradients, visualization
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")
print(sentiment_analysis("I love this!"))

[{'label': 'POSITIVE', 'score': 0.9988656044006348}]


In [3]:
model = sentiment_analysis.model
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [4]:
tokenizer = sentiment_analysis.tokenizer
tokenizer

RobertaTokenizerFast(name_or_path='siebert/sentiment-roberta-large-english', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [5]:
def predict(inputs, position_ids=None, attention_mask=None):
    output = model(inputs, position_ids=position_ids, attention_mask=attention_mask, )
    return output.logits

In [6]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [7]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + text_ids  + [sep_token_id]

    # construct reference token ids
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids

We can input text and attribution label with respect to which the attribution will be calculated.
Labels:
* 0 - negative
* 1 - positive

So, for instance, the text below has clearly negative sentiment, but we will calculate the attribution with respect to the positive label. We will obtain info which tokens contribute "anti" positive label.

In [8]:
text = "Today is a terrible day and i cant stop crying"
attribution_label = torch.tensor([[1]])

input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [9]:
scores = predict(input_ids, attention_mask=attention_mask, position_ids=position_ids)

print('Text: ', text)
print('Tokens', all_tokens)
print('Predicted Sentiment: ', scores)

Text:  Today is a terrible day and i cant stop crying
Tokens ['<s>', 'Today', 'Ġis', 'Ġa', 'Ġterrible', 'Ġday', 'Ġand', 'Ġi', 'Ġcant', 'Ġstop', 'Ġcrying', '</s>']
Predicted Sentiment:  tensor([[ 3.7684, -3.2886]], grad_fn=<AddmmBackward0>)


In [10]:
sentiment_analysis(text)

[{'label': 'NEGATIVE', 'score': 0.9992477893829346}]

In [11]:
torch.softmax(scores, 1)

tensor([[9.9914e-01, 8.6061e-04]], grad_fn=<SoftmaxBackward0>)

In [12]:
lig = LayerIntegratedGradients(predict, model.roberta.embeddings)

attributions, delta = lig.attribute(inputs=input_ids,
                                    target=attribution_label,
                                    baselines=ref_input_ids,
                                    additional_forward_args=(position_ids,attention_mask),
                                    return_convergence_delta=True)

In [13]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

attributions_sum = summarize_attributions(attributions)

In [14]:
vis = visualization.VisualizationDataRecord(
    word_attributions=attributions_sum,
    pred_prob=torch.max(torch.softmax(scores[0], dim=0)),
    pred_class=torch.argmax(scores[0]),
    true_class=torch.argmax(scores[0]),
    attr_class=str(attribution_label),
    attr_score=attributions_sum.sum(),
    raw_input_ids=all_tokens,
    convergence_score=delta)

print('\033[1m', 'Visualizations', '\033[0m')
visualization.visualize_text([vis])

 Visualizations 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (1.00),tensor([[1]]),-0.50,#s Today Ġis Ġa Ġterrible Ġday Ġand Ġi Ġcant Ġstop Ġcrying #/s


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (1.00),tensor([[1]]),-0.50,#s Today Ġis Ġa Ġterrible Ġday Ġand Ġi Ġcant Ġstop Ġcrying #/s


If we change the text, the attribution also changes.


In [15]:
text = "Today is a beautiful day and i cant stop smiling"
attribution_label = torch.tensor([[1]])

input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)
scores = predict(input_ids, attention_mask=attention_mask, position_ids=position_ids)

print('Text: ', text)
print('Tokens', all_tokens)
print('Predicted Sentiment: ', scores, " HuggingFace model:", sentiment_analysis(text))

attributions, delta = lig.attribute(inputs=input_ids,
                                    target=attribution_label,
                                    baselines=ref_input_ids,
                                    additional_forward_args=(position_ids, attention_mask),
                                    return_convergence_delta=True)

attributions_sum = summarize_attributions(attributions)
vis = visualization.VisualizationDataRecord(
    word_attributions=attributions_sum,
    pred_prob=torch.max(torch.softmax(scores[0], dim=0)),
    pred_class=torch.argmax(scores[0]),
    true_class=torch.argmax(scores[0]),
    attr_class=str(attribution_label),
    attr_score=attributions_sum.sum(),
    raw_input_ids=all_tokens,
    convergence_score=delta)

print('\033[1m', 'Visualizations', '\033[0m')
visualization.visualize_text([vis])

Text:  Today is a beautiful day and i cant stop smiling
Tokens ['<s>', 'Today', 'Ġis', 'Ġa', 'Ġbeautiful', 'Ġday', 'Ġand', 'Ġi', 'Ġcant', 'Ġstop', 'Ġsmiling', '</s>']
Predicted Sentiment:  tensor([[-3.7975,  2.9700]], grad_fn=<AddmmBackward0>)  HuggingFace model: [{'label': 'POSITIVE', 'score': 0.9988415837287903}]
 Visualizations 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),tensor([[1]]),1.93,#s Today Ġis Ġa Ġbeautiful Ġday Ġand Ġi Ġcant Ġstop Ġsmiling #/s


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
1,1 (1.00),tensor([[1]]),1.93,#s Today Ġis Ġa Ġbeautiful Ġday Ġand Ġi Ġcant Ġstop Ġsmiling #/s
